In [1]:
# %%

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import IPython
from glob import glob
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import tqdm
#import seaborn as sns
import albumentations as A
import torch
import torch.nn as nn
import timm
import torcheval 
import wandb
import torchvision
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import seaborn as sns
from albumentations.pytorch import ToTensorV2
plt.style.use('ggplot')
plt.rcParams.update(**{'figure.dpi':150})
import psutil
import imageio.v3 as imageio

# %%
RANDOM_NUMBER = 42
torch.manual_seed(RANDOM_NUMBER)

# %% [markdown]
# # select Device

# %%
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIRECTORY = os.path.join(os.getcwd(),"data")
torch.cuda.empty_cache()

# %%
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler , MinMaxScaler
import scipy as sp
#pid :279026 
# %%
class Config():

    BASE_DIR = os.path.join(os.getcwd() , 'data')
    train_df = pd.read_csv(BASE_DIR  +  '/train.csv')
    TRAIN_VAL_SPLIT_SIZE = 0.01
    TRAIN_BATCH_SIZE =2
    VAL_BATCH_SIZE = 2
    TEST_BATCH_SIZE = 2
    LR_MAX = 1e-4 
    NUM_EPOCHS = 20
    IMG_OUT_FEATURES = 768  #512  # swin     #768<-vit   
    NORMALIZE_TARGET = "log_transform_mean_std"   #"log_transform" #
    RANDOM_NUMBER = 42
    NUM_FLODS  = 5
    NUM_CLASSES = 6
    TRAITS_NAME = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean' ]
    WANDB_INIT =  False
    FOLD = 0 # Which fold to set as validation data
    IMAGE_SIZE =128
    TARGET_IMAGE_SIZE =  224
    T_MAX =        9
    torch.manual_seed(RANDOM_NUMBER)
    INCLUDE_EXTRA_FEATURES = True
    EXTRA_FEATURES_NORMALIZATION = "standard_scalar"  #"min_max_normalization"  #
    WEIGHT_DECAY = 0.01
    TABULAR_NN_OUTPUT  =   128 # 256
    IMG_MODEL_NAME = "dinoV2" #"efficientnet_v2" # 
    IMG_FINED_TUNED_WEIGHT = None  #f'{BASE_DIR}/swinV2.pth'
    Lower_Quantile = 0.005
    Upper_Quantile = 0.985 #0.985
    RECOMPUTE_IMAGE_EMBEDDINGS = True
    # 
    EXTRA_COLOUMN = ['WORLDCLIM_BIO1_annual_mean_temperature',
       'WORLDCLIM_BIO12_annual_precipitation',
       'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
       'WORLDCLIM_BIO15_precipitation_seasonality',
       'WORLDCLIM_BIO4_temperature_seasonality',
       'WORLDCLIM_BIO7_temperature_annual_range',
       'SOIL_bdod_0.5cm_mean_0.01_deg',
       'SOIL_bdod_100.200cm_mean_0.01_deg',
       'SOIL_bdod_15.30cm_mean_0.01_deg',
       'SOIL_bdod_30.60cm_mean_0.01_deg',
       'SOIL_bdod_5.15cm_mean_0.01_deg',
       'SOIL_bdod_60.100cm_mean_0.01_deg', 'SOIL_cec_0.5cm_mean_0.01_deg',
       'SOIL_cec_100.200cm_mean_0.01_deg',
       'SOIL_cec_15.30cm_mean_0.01_deg', 'SOIL_cec_30.60cm_mean_0.01_deg',
       'SOIL_cec_5.15cm_mean_0.01_deg', 'SOIL_cec_60.100cm_mean_0.01_deg',
       'SOIL_cfvo_0.5cm_mean_0.01_deg',
       'SOIL_cfvo_100.200cm_mean_0.01_deg',
       'SOIL_cfvo_15.30cm_mean_0.01_deg',
       'SOIL_cfvo_30.60cm_mean_0.01_deg',
       'SOIL_cfvo_5.15cm_mean_0.01_deg',
       'SOIL_cfvo_60.100cm_mean_0.01_deg',
       'SOIL_clay_0.5cm_mean_0.01_deg',
       'SOIL_clay_100.200cm_mean_0.01_deg',
       'SOIL_clay_15.30cm_mean_0.01_deg',
       'SOIL_clay_30.60cm_mean_0.01_deg',
       'SOIL_clay_5.15cm_mean_0.01_deg',
       'SOIL_clay_60.100cm_mean_0.01_deg',
       'SOIL_nitrogen_0.5cm_mean_0.01_deg',
       'SOIL_nitrogen_100.200cm_mean_0.01_deg',
       'SOIL_nitrogen_15.30cm_mean_0.01_deg',
       'SOIL_nitrogen_30.60cm_mean_0.01_deg',
       'SOIL_nitrogen_5.15cm_mean_0.01_deg',
       'SOIL_nitrogen_60.100cm_mean_0.01_deg',
       'SOIL_ocd_0.5cm_mean_0.01_deg', 'SOIL_ocd_100.200cm_mean_0.01_deg',
       'SOIL_ocd_15.30cm_mean_0.01_deg', 'SOIL_ocd_30.60cm_mean_0.01_deg',
       'SOIL_ocd_5.15cm_mean_0.01_deg', 'SOIL_ocd_60.100cm_mean_0.01_deg',
       'SOIL_ocs_0.30cm_mean_0.01_deg', 'SOIL_phh2o_0.5cm_mean_0.01_deg',
       'SOIL_phh2o_100.200cm_mean_0.01_deg',
       'SOIL_phh2o_15.30cm_mean_0.01_deg',
       'SOIL_phh2o_30.60cm_mean_0.01_deg',
       'SOIL_phh2o_5.15cm_mean_0.01_deg',
       'SOIL_phh2o_60.100cm_mean_0.01_deg',
       'SOIL_sand_0.5cm_mean_0.01_deg',
       'SOIL_sand_100.200cm_mean_0.01_deg',
       'SOIL_sand_15.30cm_mean_0.01_deg',
       'SOIL_sand_30.60cm_mean_0.01_deg',
       'SOIL_sand_5.15cm_mean_0.01_deg',
       'SOIL_sand_60.100cm_mean_0.01_deg',
       'SOIL_silt_0.5cm_mean_0.01_deg',
       'SOIL_silt_100.200cm_mean_0.01_deg',
       'SOIL_silt_15.30cm_mean_0.01_deg',
       'SOIL_silt_30.60cm_mean_0.01_deg',
       'SOIL_silt_5.15cm_mean_0.01_deg',
       'SOIL_silt_60.100cm_mean_0.01_deg', 'SOIL_soc_0.5cm_mean_0.01_deg',
       'SOIL_soc_100.200cm_mean_0.01_deg',
       'SOIL_soc_15.30cm_mean_0.01_deg', 'SOIL_soc_30.60cm_mean_0.01_deg',
       'SOIL_soc_5.15cm_mean_0.01_deg', 'SOIL_soc_60.100cm_mean_0.01_deg',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m9',
       'VOD_C_2002_2018_multiyear_mean_m01',
       'VOD_C_2002_2018_multiyear_mean_m02',
       'VOD_C_2002_2018_multiyear_mean_m03',
       'VOD_C_2002_2018_multiyear_mean_m04',
       'VOD_C_2002_2018_multiyear_mean_m05',
       'VOD_C_2002_2018_multiyear_mean_m06',
       'VOD_C_2002_2018_multiyear_mean_m07',
       'VOD_C_2002_2018_multiyear_mean_m08',
       'VOD_C_2002_2018_multiyear_mean_m09',
       'VOD_C_2002_2018_multiyear_mean_m10',
       'VOD_C_2002_2018_multiyear_mean_m11',
       'VOD_C_2002_2018_multiyear_mean_m12',
       'VOD_Ku_1987_2017_multiyear_mean_m01',
       'VOD_Ku_1987_2017_multiyear_mean_m02',
       'VOD_Ku_1987_2017_multiyear_mean_m03',
       'VOD_Ku_1987_2017_multiyear_mean_m04',
       'VOD_Ku_1987_2017_multiyear_mean_m05',
       'VOD_Ku_1987_2017_multiyear_mean_m06',
       'VOD_Ku_1987_2017_multiyear_mean_m07',
       'VOD_Ku_1987_2017_multiyear_mean_m08',
       'VOD_Ku_1987_2017_multiyear_mean_m09',
       'VOD_Ku_1987_2017_multiyear_mean_m10',
       'VOD_Ku_1987_2017_multiyear_mean_m11',
       'VOD_Ku_1987_2017_multiyear_mean_m12',
       'VOD_X_1997_2018_multiyear_mean_m01',
       'VOD_X_1997_2018_multiyear_mean_m02',
       'VOD_X_1997_2018_multiyear_mean_m03',
       'VOD_X_1997_2018_multiyear_mean_m04',
       'VOD_X_1997_2018_multiyear_mean_m05',
       'VOD_X_1997_2018_multiyear_mean_m06',
       'VOD_X_1997_2018_multiyear_mean_m07',
       'VOD_X_1997_2018_multiyear_mean_m08',
       'VOD_X_1997_2018_multiyear_mean_m09',
       'VOD_X_1997_2018_multiyear_mean_m10',
       'VOD_X_1997_2018_multiyear_mean_m11',
       'VOD_X_1997_2018_multiyear_mean_m12'
       ]

    N_TARGETS  =len(TRAITS_NAME)  


CONFIG = Config()



# %%
if CONFIG.WANDB_INIT:
    wandb.login()
    wandb.init(project="cs680v3",group="dino_V2",name="submission_dinoV2_no_fine_tune",
            config = {
        "LR_max": CONFIG.LR_MAX,
        "WEIGHT_DECAY":CONFIG.WEIGHT_DECAY,
        "train_batch" : CONFIG.TRAIN_BATCH_SIZE
        })


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:

# %% [markdown]
# # Preprocessing the Tabular Data And Image Transformation

# %%
# define_transformation for the tabular data
log_tf_col = CONFIG.TRAITS_NAME
scale_feature_col =  CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME
log_transform = ColumnTransformer(transformers= [
    ('log' , FunctionTransformer( np.log10  , inverse_func=sp.special.exp10, validate=False, check_inverse = True ,feature_names_out='one-to-one') , log_tf_col)
        ] , verbose_feature_names_out=False ,remainder= 'passthrough'
          )
log_transform.set_output(transform='pandas')

std_scale =  ColumnTransformer(transformers=[('scale',StandardScaler() ,scale_feature_col )  ],
                               verbose_feature_names_out=False,
                               remainder='passthrough'
                               )
std_scale.set_output(transform='pandas')
scaling_pipeline = Pipeline(steps=[   
                        ("log" , log_transform),
                        ("std_scale" , std_scale )])



# %%
# preparing the tabular data that has been given 
BASE_DIR = CONFIG.BASE_DIR
Train_DF = pd.read_csv(BASE_DIR  +  '/train.csv')
Test_DF =  pd.read_csv(BASE_DIR  +  '/test.csv')
Test_DF[log_tf_col] =1

train_df , val_df = train_test_split(Train_DF,test_size=CONFIG.TRAIN_VAL_SPLIT_SIZE,shuffle=True)
for column in CONFIG.TRAITS_NAME:
    lower_quantile = train_df[column].quantile(CONFIG.Lower_Quantile)
    upper_quantile = train_df[column].quantile(CONFIG.Upper_Quantile)
    train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

# train_df = train_df[(np.abs(stats.zscore(np.log10(train_df[CONFIG.TRAITS_NAME]))<2).all(axis=1)  )]

print('JPEG Files Processing:')
train_df['file_path'] = train_df['id'].apply(lambda s: f'{BASE_DIR}/train_images/{s}.jpeg')
train_df['jpeg_bytes'] = train_df['file_path'].apply(lambda fp: open(fp, 'rb').read())

val_df['file_path'] = val_df['id'].apply(lambda s: f'{BASE_DIR}/train_images/{s}.jpeg')
val_df['jpeg_bytes'] = val_df['file_path'].apply(lambda fp: open(fp, 'rb').read())

Test_DF['file_path'] = Test_DF['id'].apply(lambda s: f'{BASE_DIR}/test_images/{s}.jpeg')
Test_DF['jpeg_bytes'] = Test_DF['file_path'].apply(lambda fp: open(fp, 'rb').read())
print('JPEG Files Processing End')    


# train_tabular = train_df.drop(columns = ['id'] + CONFIG.TRAITS_NAME)
# val_tabular =    val_df.drop(columns = ['id'] + CONFIG.TRAITS_NAME)
# test_tabular = test.drop(columns = ['id'])

# scaling of the training data !

training_scaling_pipeline = Pipeline(steps=[   
                        ("log" , log_transform),
                        ("std_scale" , std_scale )])

train_tabular_scaled =  training_scaling_pipeline.fit_transform(train_df)
validation_tabular_scaled = training_scaling_pipeline.transform(val_df)

test_tabular_scaled = training_scaling_pipeline.transform(Test_DF)
Test_DF[log_tf_col]
"inverse"
# tt =   scaling_pipeline['std_scale']['scale'].inverse_transform(train_tabular_scaled[CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME])
# tt = pd.DataFrame(tt, columns=CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME)
# tt2 = scaling_pipeline['log']['log'].inverse_transform(tt[CONFIG.TRAITS_NAME])
# tt2  = pd.concat([tt2 ,tt[CONFIG.EXTRA_COLOUMN]],axis=1)                 
                  


# some hyper parameter for lr scheduler
CONFIG.N_TRAIN_SAMPLES = len(train_df)
CONFIG.N_STEPS_PER_EPOCH = (CONFIG.N_TRAIN_SAMPLES // CONFIG.TRAIN_BATCH_SIZE)
CONFIG.N_STEPS = CONFIG.N_STEPS_PER_EPOCH * CONFIG.NUM_EPOCHS + 1   

# %%
# Image PreProcessing

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
TRAIN_TRANSFORMS = A.Compose([
    A.Resize(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    A.RandomResizedCrop(size=(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE), interpolation=cv2.INTER_CUBIC,p=0.4),  # Simulate different crops
    A.HorizontalFlip(p=0.2),  
    A.ToFloat(),
    A.Normalize(mean=MEAN, std=STD, max_pixel_value=1),
    ToTensorV2(),
])

TEST_TRANSFORMS = A.Compose([
    A.Resize(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    #A.CenterCrop(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    A.ToFloat(),
    A.Normalize(mean=MEAN, std=STD, max_pixel_value=1),
    ToTensorV2(),
])



# %% [markdown]
# # Creating the Dataloader for Train , Validation and Testing 

# %%
class create_dataset(Dataset):
    def __init__(self, X_jpeg_bytes, X_tabular, y, transforms=None):
        self.X_jpeg_bytes = X_jpeg_bytes
        self.X_tabular = X_tabular
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        X_sample = self.transforms(
            image=imageio.imread(self.X_jpeg_bytes[index]),
        )['image']
        X_tabular_sample = self.X_tabular[index]
        y_sample = self.y[index]

        return X_sample, X_tabular_sample, y_sample




train_dataset = create_dataset(
    X_jpeg_bytes = train_tabular_scaled['jpeg_bytes'].values,
    X_tabular = train_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = train_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32),
    transforms= TRAIN_TRANSFORMS
)

validation_dataset = create_dataset(
    X_jpeg_bytes = validation_tabular_scaled['jpeg_bytes'].values,
    X_tabular = validation_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = validation_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32),
    transforms= TEST_TRANSFORMS
)

test_dataset = create_dataset(
    X_jpeg_bytes = test_tabular_scaled['jpeg_bytes'].values,
    X_tabular = test_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = test_tabular_scaled["id"].values,
    transforms= TEST_TRANSFORMS
    )

train_dataloader = DataLoader(
    train_dataset,
    batch_size=CONFIG.TRAIN_BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=0#psutil.cpu_count(),
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=CONFIG.VAL_BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    num_workers= 0 #psutil.cpu_count(),
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=CONFIG.TEST_BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    num_workers= 0 #psutil.cpu_count(),
)

JPEG Files Processing:
JPEG Files Processing End


In [3]:
if CONFIG.RECOMPUTE_IMAGE_EMBEDDINGS:
    class ImageBackbone_dinoV2(nn.Module):
        def __init__(self, backbone_name, weight_path, out_features, fixed_feature_extractor=False):
            super().__init__()
            self.out_features = out_features
            self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg').to(DEVICE)
            in_features = self.backbone.num_features
            self.backbone.head = nn.Identity()
            self.head = nn.Sequential(
                nn.Linear(in_features, out_features),
            )

        def forward(self, x):
            x = self.backbone(x)
            return self.head(x)

    class TabularBackbone(nn.Module):
        def __init__(self, n_features, out_features):
            super().__init__()
            self.out_features = out_features
            self.fc = nn.Sequential(
                nn.Linear(n_features, 512),
                nn.BatchNorm1d(512),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(512, out_features),
            )

        def forward(self, x):
            return self.fc(x)    



    # %% [markdown]
    # # Combine the image feature extraction model with tabular feature extraction model 




    #backbone_name, weight_path, out_features, fixed_feature_extractor=False
    # %%
    class CustomModel(nn.Module):
        def __init__(self,model_name , fine_tuned_img_weights , img_out_features , fixed_feature_extractor , traits_target_feature):
            super().__init__()
            self.img_backbone =  ImageBackbone_dinoV2(backbone_name = model_name,weight_path = fine_tuned_img_weights ,out_features =  img_out_features ,fixed_feature_extractor=fixed_feature_extractor)
            self.extra_features_model = TabularBackbone(n_features  = len(CONFIG.EXTRA_COLOUMN) , out_features=128)
            self.fc = nn.Sequential(
                nn.Linear(self.extra_features_model.out_features + self.img_backbone.out_features,1024 ),  # 256
                nn.BatchNorm1d(1024),
                nn.GELU(),
                # nn.Dropout(0.1),
                nn.Linear(1024, 256),
                nn.BatchNorm1d(256),
                nn.GELU(),
                # nn.Dropout(0.1),
                nn.Linear(256, traits_target_feature),
            )        
        def forward(self,image,x):
            output_image = self.img_backbone(image) 
            z = self.extra_features_model(x) 
            inputs  = torch.cat((output_image,z), 1 )
            output = self.fc(inputs)
            return output

    # input_channels = len(CONFIG.EXTRA_COLOUMN) ,out_channels =CONFIG.TABULAR_NN_OUTPUT, target_features_num= len(CONFIG.TRAITS_NAME), tim_num_class=CONFIG.TIM_NUM_CLASS , model_name=CONFIG.TIM_MODEL_NAME
    # %%
    model = CustomModel(model_name=CONFIG.IMG_MODEL_NAME , fine_tuned_img_weights= CONFIG.IMG_FINED_TUNED_WEIGHT,img_out_features = CONFIG.IMG_OUT_FEATURES, fixed_feature_extractor=True,traits_target_feature = CONFIG.N_TARGETS )

    



Using cache found in /home/prajwal/.cache/torch/hub/facebookresearch_dinov2_main
/home/prajwal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/prajwal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/prajwal/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
INFO:dinov2:using SwiGLU layer as FFN


In [4]:
if CONFIG.RECOMPUTE_IMAGE_EMBEDDINGS:
    model.load_state_dict(torch.load('/home/prajwal/cs680/cs680_kaggle/data/submission_dinoV2_not_fine_tuned.pth'))
    model.eval()
    model.to(DEVICE) 


/tmp/ipykernel_68537/1428918084.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/prajwal/cs680/cs680_kaggle/data/submission_dinoV2

In [5]:
# X_img_test , X_features,targets= next(iter(train_dataloader))
# x_f = X_features.to(DEVICE);
# x_img =X_img_test.to(DEVICE);
# output = model(x_img,x_f)
# activation['fc.3']

In [6]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.fc[3].register_forward_hook((get_activation('fc.3')))

# catbooster

In [7]:
def get_image_embeddings_dinoV2(batch_image , batch_features,batch_size= CONFIG.TRAIN_BATCH_SIZE):
    output = model(batch_image,batch_features);
    curr_feature_embeddings=activation['fc.3']
    feature_embeddings = (curr_feature_embeddings.cpu().numpy())
    return feature_embeddings

In [8]:
SAVE_DIR = "/home/prajwal/cs680/cs680_kaggle/catboost_dinoV2_features/"
suffix = 'dinoV2_embs_reg'
if CONFIG.RECOMPUTE_IMAGE_EMBEDDINGS:
    train_feat_embeddings_list = []
    val_feat_embeddings_list   = []
    test_feat_embeddings_list =  []
    with tqdm.tqdm(total=len(train_dataloader))as trainingLoop  :
        for index,batch in enumerate(iter(train_dataloader)) :
            X_train_img = batch[0].to(DEVICE) 
            X_train_feat  = batch[1].to(DEVICE)
            X_train_id  = batch[2].to(DEVICE)
            train_image_embeddings = get_image_embeddings_dinoV2(X_train_img, X_train_feat)
            trainingLoop.set_description(f"in training Batch: {index}/{len(train_dataloader)} ")
            train_feat_embeddings_list.extend(train_image_embeddings)
        np.save(f'{SAVE_DIR}/train_{suffix}', np.array(train_feat_embeddings_list))
        
    with tqdm.tqdm(total=len(validation_dataloader)) as valloop  :
        for index,batch in enumerate(iter(validation_dataloader)) :
            X_train_img = batch[0].to(DEVICE) 
            X_train_feat  = batch[1].to(DEVICE)
            X_train_id  = batch[2].to(DEVICE)
            val_image_embeddings = get_image_embeddings_dinoV2(X_train_img, X_train_feat)
            valloop.set_description(f"IN val Batch: {index}/{len(validation_dataloader)} ")
            val_feat_embeddings_list.extend(val_image_embeddings)
        np.save(f'{SAVE_DIR}/val_{suffix}', np.array(val_feat_embeddings_list))
    
    with tqdm.tqdm(total=len(test_dataloader)) as testloop :
        for index,batch in enumerate(iter(test_dataloader)) :
            X_train_img = batch[0].to(DEVICE) 
            X_train_feat  = batch[1].to(DEVICE)
            X_train_id  = batch[2].to(DEVICE)
            test_image_embeddings = get_image_embeddings_dinoV2(X_train_img, X_train_feat)
            testloop.set_description(f"IN test Batch: {index}/{len(test_dataloader)} ")
            test_feat_embeddings_list.extend(test_image_embeddings)
        np.save(f'{SAVE_DIR}/test_{suffix}', np.array(test_feat_embeddings_list))       
else:
    suffix = 'image_embs_dinov2_vitg14_reg'
    train_image_embeddings = np.load(f'{SAVE_DIR}/train_{suffix}.npy')
    val_image_embeddings = np.load(f'{SAVE_DIR}/val_{suffix}.npy')
    test_image_embeddings = np.load(f'{SAVE_DIR}/test_{suffix}.npy')
    print(f'Embeddings {suffix} loaded from dataset.')

IN test Batch: 3195/3196 :   0%|          | 0/3196 [06:11<?, ?it/s]


In [21]:
i

6391

In [13]:
import os

import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from catboost import Pool, CatBoostRegressor
from torchvision import transforms


In [14]:
X_train_tabular = train_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32)
y_train_tabular = train_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32)

X_val_tabular = validation_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32)
y_val_tabular = validation_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32)

X_test_tabular = test_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32)
y_test_tabular = test_tabular_scaled["id"].values


In [16]:
t = PolynomialFeatures(2).fit_transform(X_train_tabular)[:, :first_n_poly_feats]

In [32]:
t.shape

(38032, 1000)

In [34]:
train_feat_embeddings_array = np.empty((0,256))
val_feat_embeddings_array = np.empty((0,256))
test_feat_embeddings_array = np.empty((0,256))
for i in range(len(train_feat_embeddings_list)):
    #print(i)
    train_feat_embeddings_array= np.vstack((train_feat_embeddings_array ,train_feat_embeddings_list[i] ))
    
for i in range(len(val_feat_embeddings_list)):
    #print(i)
    val_feat_embeddings_array= np.vstack((val_feat_embeddings_array ,val_feat_embeddings_list[i] ))
    
for i in range(len(test_feat_embeddings_list)):
    #print(i)
    test_feat_embeddings_array= np.vstack((test_feat_embeddings_array ,test_feat_embeddings_list[i] ))

In [36]:
np.save(f'{SAVE_DIR}/train_{suffix}_array', np.array(train_feat_embeddings_array))
np.save(f'{SAVE_DIR}/val_{suffix}_array', np.array(val_feat_embeddings_array))
np.save(f'{SAVE_DIR}/test_{suffix}_array', np.array(test_feat_embeddings_array))

In [37]:
# we can potentially use all the polynomial features but it would take an etenriny to train the models
first_n_poly_feats = 1000

#drop images and id
train_features_mask_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(X_train_tabular)[:, :first_n_poly_feats], train_feat_embeddings_array), axis=1
)
val_features_mask_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(X_val_tabular)[:, :first_n_poly_feats], val_feat_embeddings_array), axis=1
)
test_features_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(X_test_tabular)[:, :first_n_poly_feats], test_feat_embeddings_array), axis=1
)

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
        1247,  1248,  1249,  1250,  1251,  1252,  1253,  1254,  1255, 'emb'],
      dtype='object', length=1257)

In [45]:
train_features_mask_df = pd.DataFrame(train_features_mask_all)
train_features_mask_df['emb'] = list(train_feat_embeddings_list)

val_features_mask_df = pd.DataFrame(val_features_mask_all)
val_features_mask_df['emb'] = list(val_feat_embeddings_list)

test_features_mask_df = pd.DataFrame(test_features_all)
test_features_mask_df['emb'] = list(test_feat_embeddings_list)

In [47]:
y_val_tabular.shape

(434, 6)

In [50]:
models = {}
scores = {}
models = {}
scores = {}
for i, col in tqdm(enumerate(CONFIG.TRAITS_NAME), total=len(CONFIG.TRAITS_NAME)):
    y_curr = y_train_tabular[:, i]
    y_curr_val = y_val_tabular[:, i]
    train_pool = Pool(train_features_mask_df, y_curr, embedding_features=['emb'])
    val_pool = Pool(val_features_mask_df, y_curr_val, embedding_features=['emb'])
    
    # tried to tune these parameters but without real success 
    model = CatBoostRegressor(iterations=1500, learning_rate=0.06, loss_function='RMSE', verbose=1, random_state=CONFIG.RANDOM_NUMBER)
    model.fit(train_pool)
    models[col] = model
    
    y_curr_val_pred = model.predict(val_pool)
    
    r2_col = r2_score(y_curr_val, y_curr_val_pred)
    scores[col] = r2_col
    print(f'Target: {col}, R2: {r2_col:.3f}')
# this val score somewhat correlates with submission score bit I didn't really bother
print(f'Mean R2: {np.mean(list(scores.values())):.3f}')

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
submission = pd.DataFrame({'id': test_features_mask_df['id']})
submission[CONFIG.TRAITS_NAME] = 0
submission.columns = submission.columns.str.replace('_mean', '')

In [ ]:
for i, col in enumerate(CONFIG.TARGET_COLUMNS):
    test_pool = Pool(test_features_mask_df, embedding_features=['emb'])
    col_pred = models[col].predict(test_pool)
    submission[col.replace('_mean', '')] = col_pred

submission.to_csv('submission.csv', index=False)
submission.head()

In [9]:
# # to save model 
# class BestModelSaveCallback:
#     def __init__(self, save_path):
#         self.save_path = save_path
#         self.best_accuracy = -1

#     def __call__(self, accuracy,model):
#         if accuracy > self.best_accuracy:
#             self.best_accuracy = accuracy
#             model.to(device = "cpu")
#             torch.save(model.state_dict(), self.save_path)
#             model.to(device=DEVICE)
            
#             print("generating result on test data")
#             submission_df = pd.DataFrame(columns=CONFIG.TRAITS_NAME)
#             for index , batch in tqdm.tqdm(enumerate(iter(test_dataloader))):
#                 X_img_test = batch[0] 
#                 X_features  = batch[1]
#                 test_id  = batch[2]
#                 #print(batch) 
#                 with torch.no_grad():
#                     #print(X_img_test.shape, X_features.shape)
#                     y_pred = model(X_img_test.to(DEVICE),X_features.to(DEVICE)).detach().cpu().numpy()  #,X_features.to(DEVICE)
                
#                     pred_pd = pd.DataFrame(columns=CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME)
#                     pred_pd[CONFIG.EXTRA_COLOUMN] =-1
#                     pred_pd[CONFIG.TRAITS_NAME] = y_pred 

#                     temp1 =   scaling_pipeline['std_scale']['scale'].inverse_transform(pred_pd)
#                     temp2=    pd.DataFrame(temp1, columns=CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME)
#                     pred_final =   scaling_pipeline['log']['log'].inverse_transform(temp2[CONFIG.TRAITS_NAME])
#                     #pred_final["id"] = test_id.cpu().detach().numpy()
#                     submission_df = pd.concat([submission_df, pred_final.assign(id=test_id.cpu().detach().numpy())], ignore_index=True)
#             # submission_df.to_csv('submission_self_tuning.csv', index=False)
#             submission_df[["id"]  + CONFIG.TRAITS_NAME ].to_csv('submission_dinoV2_not_fine_tuned.csv', index=False)
#             print("Submit!")